In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd
from tqdm.auto import tqdm
import time

data = pd.DataFrame(columns=['Gospodarze','Goście', 'Posiadanie piłki H', 'Posiadanie piłki A', 'Sytuacje bramkowe H', 'Sytuacje bramkowe A', 'Strzały na bramkę H', 'Strzały na bramkę A', 'Strzały niecelne H', 'Strzały niecelne A', 'Strzały zablokowane H', 'Strzały zablokowane A', 'Rzuty wolne H', 'Rzuty wolne A', 'Rzuty rożne H', 'Rzuty rożne A', 'Spalone H', 'Spalone A', 'Wrzuty z autu H', 'Wrzuty z autu A', 'Interwencje bramkarzy H', 'Interwencje bramkarzy A', 'Faule H', 'Faule A', 'Czerwone kartki H', 'Czerwone kartki A', 'Żółte kartki H', 'Żółte kartki A', 'Sezon'])

options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

# Otwieramy stronę z danymi meczu
for iter in tqdm(range(2013,2022,1)):
    driver.get(f"https://www.flashscore.pl/pilka-nozna/polska/pko-bp-ekstraklasa-{iter}-{iter+1}/wyniki/")
    # Czekamy, aż elementy zostaną załadowane
    wait = WebDriverWait(driver, 10)
    while True:
        try:
            more_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "event__more--static")))
            more_button.click()
            time.sleep(5)
        except:
            break

    aaa = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "event__match")))
    pbar = tqdm(total=len(aaa))

    for match in wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "event__match"))):
        match.click()
        wait.until(EC.number_of_windows_to_be(2))
        driver.switch_to.window(driver.window_handles[1])
        stats_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[href='#/szczegoly-meczu/statystyki-meczu']")))
        stats_button.click()
        categories = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "stat__row")))
        team_names = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".participant__participantName")))

        row = ['Not found'] * 29
        row[0] = team_names[1].text
        row[1] = team_names[2].text

        for i in categories:
            i = i.text.split('\n')
            match i[1]:
                case 'Posiadanie piłki':
                    row[2] = i[0]
                    row[3] = i[2]
                case 'Sytuacje bramkowe':
                    row[4] = i[0]
                    row[5] = i[2]
                case 'Strzały na bramkę':
                    row[6] = i[0]
                    row[7] = i[2]
                case 'Strzały niecelne':
                    row[8] = i[0]
                    row[9] = i[2]         
                case 'Strzały zablokowane':
                    row[10] = i[0]
                    row[11] = i[2]
                case 'Rzuty wolne':
                    row[12] = i[0]
                    row[13] = i[2]
                case 'Rzuty rożne':
                    row[14] = i[0]
                    row[15] = i[2]  
                case 'Spalone':
                    row[16] = i[0]
                    row[17] = i[2]    
                case 'Wrzuty z autu':
                    row[18] = i[0]
                    row[19] = i[2]   
                case 'Interwencje bramkarzy':
                    row[20] = i[0]
                    row[21] = i[2]   
                case 'Faule':
                    row[22] = i[0]
                    row[23] = i[2]   
                case 'Czerwone kartki':
                    row[24] = i[0]
                    row[25] = i[2]       
                case 'Żółte kartki':
                    row[26] = i[0]
                    row[27] = i[2] 

        row[28] = f'{iter}/{iter+1}'
        print(row)
        row=pd.DataFrame([row], columns=['Gospodarze','Goście', 'Posiadanie piłki H', 'Posiadanie piłki A', 'Sytuacje bramkowe H', 'Sytuacje bramkowe A', 'Strzały na bramkę H', 'Strzały na bramkę A', 'Strzały niecelne H', 'Strzały niecelne A', 'Strzały zablokowane H', 'Strzały zablokowane A', 'Rzuty wolne H', 'Rzuty wolne A', 'Rzuty rożne H', 'Rzuty rożne A', 'Spalone H', 'Spalone A', 'Wrzuty z autu H', 'Wrzuty z autu A', 'Interwencje bramkarzy H', 'Interwencje bramkarzy A', 'Faule H', 'Faule A', 'Czerwone kartki H', 'Czerwone kartki A', 'Żółte kartki H', 'Żółte kartki A', 'Sezon'])

        #data.loc[len(data)] = row

        data = pd.concat([data,row])
        data.to_excel('output.xlsx', index=False)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        pbar.update(1)




# Zamykamy przeglądarkę
driver.quit()
